# JetBlue Route Planning – Python Analysis

# Executive Summary

Using a mock dataset created for educational and portfolio purposes, this project models JetBlue route performance through a realistic, end‑to‑end analytics workflow that mirrors the responsibilities of a Route Planning Analyst. The analysis combines Python, SQL, and Tableau to evaluate financial performance, operational capacity, and month‑over‑month growth across a set of representative JetBlue routes.

Key enhancements include:
- Realistic aircraft assignments (A220, A320, A321, A321 Mint) based on JetBlue’s
  actual fleet deployment patterns.
- Seat configuration mapping to support capacity and load factor-style analysis.
- Financial metrics such as revenue, cost, profit, and profit margin.
- Time-series metrics including lagged values and month-over-month growth.
- SQL queries that replicate core analytical tasks used in airline planning.
- Tableau dashboards that visualize route profitability, growth trends, and
  aircraft-level performance.

This notebook demonstrates the ability to structure data, build analytical models,
communicate insights, and create executive-ready visualizations—key skills for
aviation analytics roles.

# Table of Contents

1. [Section 1 — Imports & Environment Setup](#section-1--imports--environment-setup)
2. [Section 2 — Load & Inspect Raw Dataset](#section-2--load--inspect-raw-dataset)
3. [Section 3 — Data Enrichment: Aircraft Types, Seat Configuration, and Fleet Utilization](#section-3--data-enrichment-aircraft-types-seat-configuration-and-fleet-utilization)
4. [Section 4 — Financial Metrics (Revenue, Cost, Profit, Margin)](#section-4--financial-metrics-revenue-cost-profit-margin)
5. [Section 5 — Growth Metrics (Lag, MoM Growth)](#section-5--growth-metrics-lag-mom-growth)
6. [Section 6 — Export Enriched Dataset for Tableau](#section-6--export-enriched-dataset-for-tableau)
7. [Section 7 — Tableau Dashboards](#section-7--tableau-dashboards)
8. [Section 8 — SQL Analysis](#section-8--sql-analysis)
9. [Section 9 — Conclusion & Recommendations](#section-9--conclusion--recommendations)


# Section 1 — Imports & Environment Setup

This section initializes the Python environment and loads all required libraries for
data manipulation, visualization, and SQL integration. These imports support the full
analytics workflow, including data enrichment, financial modeling, growth analysis,
and exporting the final dataset for Tableau.

Key libraries include:
- **pandas** for data manipulation
- **numpy** for numerical operations
- **sqlite3** for SQL queries within the notebook
- **matplotlib / seaborn** (optional) for quick visual checks

In [32]:
import pandas as pd
import numpy as np
import sqlite3

pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [33]:
import sys
sys.executable

'c:\\Users\\k_daw\\OneDrive\\jetblue-route-planning\\.venv\\Scripts\\python.exe'

# Section 2 — Load & Inspect Raw Dataset

The raw JetBlue route performance dataset is loaded into a pandas DataFrame for
initial inspection. The goal of this section is to verify that the data is structured
correctly and ready for enrichment in later sections.

Initial checks include:
- Column names and data types  
- Number of rows and columns  
- Sample records  
- Basic descriptive statistics  

These checks ensure the dataset is clean and suitable for downstream financial,
growth, and fleet-level analysis.

## 2.1 Load Raw Dataset

We load the raw `jetblue_route_performance_enriched.csv` file and perform basic structural
checks to confirm the dataset is ready for enrichment.

In [34]:
# Load raw data
df = pd.read_csv("route_monthly_performance.csv")

# Quick inspection
df.head()
df.info()
df.describe()

<class 'pandas.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   origin            120 non-null    str    
 1   destination       120 non-null    str    
 2   month             120 non-null    str    
 3   distance_miles    120 non-null    int64  
 4   seats             120 non-null    int64  
 5   passengers        120 non-null    int64  
 6   load_factor       120 non-null    float64
 7   avg_fare          120 non-null    int64  
 8   competitor_seats  120 non-null    int64  
dtypes: float64(1), int64(5), str(3)
memory usage: 8.6 KB


,distance_miles,seats,passengers,load_factor,avg_fare,competitor_seats
count,120.00,120.00,120.00,120.00,120.00,120.00
mean,"1,738.40","11,450.00","10,392.92",0.91,177.51,"11,933.33"
std,671.47,"2,205.23","2,082.52",0.05,29.35,"4,940.37"
min,816.00,"8,000.00","6,700.00",0.82,125.00,"3,000.00"
25%,"1,104.00","9,500.00","8,700.00",0.87,151.50,"7,500.00"
50%,"1,637.50","11,500.00","10,200.00",0.91,178.00,"13,500.00"
75%,"2,475.00","13,000.00","12,050.00",0.95,198.50,"16,000.00"
max,"2,586.00","15,000.00","14,800.00",0.99,245.00,"20,000.00"


# Section 3 — Data Enrichment: Aircraft Types, Seat Configuration, and Fleet Utilization

This section enhances the dataset with operational attributes that reflect JetBlue’s
real-world fleet deployment. Each route is assigned an aircraft type based on typical
JetBlue scheduling patterns (A220, A320, A321, A321 Mint), and seat configurations are
mapped accordingly. These additions enable capacity-aware profitability and utilization
analysis.

Enhancements include:
- **Aircraft Type Assignment** based on JetBlue’s real-world fleet usage
- **Seat Configuration Mapping** to support load factor-style analysis
- **Fleet Utilization Summary** showing passengers, capacity, and load proxy by aircraft type

These operational enrichments provide the foundation for evaluating route performance
in a realistic airline planning context.

### 3.0 Prepare Dataset for Enrichment

Before adding aircraft attributes and fleet utilization metrics, the dataset is cleaned
and ordered to ensure consistent time-series behavior. This includes converting the
month column to a datetime format and sorting the data by origin, destination, and
month.

In [35]:
df["month"] = pd.to_datetime(df["month"])
df.sort_values(["origin", "destination", "month"], inplace=True)
df.head()

,origin,destination,month,distance_miles,seats,passengers,load_factor,avg_fare,competitor_seats
72,BOS,CHS,2024-01-01,816,8000,6900,0.86,129,3000
73,BOS,CHS,2024-02-01,816,8000,6700,0.84,125,3000
74,BOS,CHS,2024-03-01,816,8000,7200,0.90,135,3500
75,BOS,CHS,2024-04-01,816,8000,7300,0.91,138,3500
76,BOS,CHS,2024-05-01,816,8000,7600,0.95,145,4000


### 3.1 Aircraft Type Assignment

Each route is assigned a JetBlue aircraft type based on typical fleet deployment
patterns. These assignments introduce operational realism and enable aircraft-level
profitability and utilization analysis.

Assignment logic:
- **A321 Mint** on premium transcontinental routes (JFK–LAX, JFK–SFO, JFK–SAN)  
- **A321 / A320** on high-demand medium/long-haul domestic routes  
- **A220** on shorter or developing routes where right-sizing capacity is important  

These choices reflect how JetBlue deploys its fleet in practice: A321 aircraft on
premium, high-yield routes; A320s on core domestic markets; and A220s on thinner or
emerging routes.

In [36]:
# Create route identifier for aircraft assignment
df["route"] = df["origin"] + "-" + df["destination"]

# Aircraft assignment based on realistic JetBlue fleet usage
aircraft_map = {
    "JFK-LAX": "A321 Mint",
    "JFK-SFO": "A321 Mint",
    "JFK-SAN": "A321 Mint",
    "BOS-SEA": "A321",
    "BOS-DEN": "A320",
    "BOS-MCO": "A320",
    "BOS-CHS": "A220",
    "FLL-AUS": "A220",
    "FLL-EWR": "A220",
    "JFK-AUS": "A320"
}

df["aircraft_type"] = df["route"].map(aircraft_map)
df.head()

,origin,destination,month,distance_miles,seats,passengers,load_factor,avg_fare,competitor_seats,route,aircraft_type
72,BOS,CHS,2024-01-01,816,8000,6900,0.86,129,3000,BOS-CHS,A220
73,BOS,CHS,2024-02-01,816,8000,6700,0.84,125,3000,BOS-CHS,A220
74,BOS,CHS,2024-03-01,816,8000,7200,0.90,135,3500,BOS-CHS,A220
75,BOS,CHS,2024-04-01,816,8000,7300,0.91,138,3500,BOS-CHS,A220
76,BOS,CHS,2024-05-01,816,8000,7600,0.95,145,4000,BOS-CHS,A220


### 3.2 Seat Configuration Mapping

Seat configurations are mapped to each aircraft type to support capacity-aware
analysis. These values approximate JetBlue’s real-world seating layouts and enable
load factor-style calculations in later sections.

In [37]:
seat_map = {
    "A220": 140,
    "A320": 162,
    "A321": 200,
    "A321 Mint": 159  # 16 Mint + 143 economy
}

df["seats_configured"] = df["aircraft_type"].map(seat_map)

In [38]:
df[["route", "aircraft_type", "seats_configured"]].drop_duplicates()

,route,aircraft_type,seats_configured
72,BOS-CHS,A220,140
108,BOS-DEN,A320,162
24,BOS-MCO,A320,162
36,BOS-SEA,A321,200
48,FLL-AUS,A220,140
84,FLL-EWR,A220,140
60,JFK-AUS,A320,162
0,JFK-LAX,A321 Mint,159
96,JFK-SAN,A321 Mint,159
12,JFK-SFO,A321 Mint,159


### 3.3 Fleet Utilization Summary

To complement the aircraft assignment and seat configuration enhancements, this section
provides a high-level view of how each aircraft type is utilized across the network.
Fleet utilization is a core component of airline planning, as it connects demand,
capacity, and profitability.

This summary highlights:
- Total passengers carried by each aircraft type  
- Average passengers per flight (a proxy for demand)  
- Configured seat capacity  
- A simplified load factor proxy (average passengers ÷ seats configured)  

While not a full operational utilization model, this provides a realistic snapshot of
how JetBlue’s fleet performs across the selected routes.

In [39]:
fleet_summary = (
    df.groupby("aircraft_type")
      .agg(
          total_passengers=("passengers", "sum"),
          avg_passengers_per_flight=("passengers", "mean"),
          seats_configured=("seats_configured", "first")
      )
      .reset_index()
)

fleet_summary

,aircraft_type,total_passengers,avg_passengers_per_flight,seats_configured
0,A220,291500,"8,097.22",140
1,A320,367550,"10,209.72",162
2,A321,127000,"10,583.33",200
3,A321 Mint,461100,"12,808.33",159


In [40]:
fleet_summary["avg_load_factor_proxy"] = (
    fleet_summary["avg_passengers_per_flight"] / fleet_summary["seats_configured"]
)

fleet_summary

,aircraft_type,total_passengers,avg_passengers_per_flight,seats_configured,avg_load_factor_proxy
0,A220,291500,"8,097.22",140,57.84
1,A320,367550,"10,209.72",162,63.02
2,A321,127000,"10,583.33",200,52.92
3,A321 Mint,461100,"12,808.33",159,80.56


# Section 4 — Financial Metrics (Revenue, Cost, Profit, Margin)

This section enriches the dataset with core financial metrics used in airline route
profitability analysis. These metrics quantify the financial performance of each
route-month combination and form the basis for evaluating market strength and
identifying opportunities for optimization.

Metrics added:
- **Revenue** = passengers × average fare  
- **Cost** = passenger-related variable costs + fixed flight operating costs  
- **Profit** = revenue − cost  
- **Profit Margin** = profit ÷ revenue  

These financial metrics provide a clear view of route-level performance and support
subsequent growth analysis and opportunity scoring.

### 4.1 Revenue Calculation

Revenue represents the total amount generated from ticket sales for each route-month.
It is calculated as:

**Revenue = Passengers × Average Fare**

This metric provides the top-line financial performance of each route and is essential
for understanding demand and pricing effectiveness.

In [41]:
df["revenue"] = df["passengers"] * df["avg_fare"]
df[["route", "month", "passengers", "avg_fare", "revenue"]].head()

,route,month,passengers,avg_fare,revenue
72,BOS-CHS,2024-01-01,6900,129,890100
73,BOS-CHS,2024-02-01,6700,125,837500
74,BOS-CHS,2024-03-01,7200,135,972000
75,BOS-CHS,2024-04-01,7300,138,1007400
76,BOS-CHS,2024-05-01,7600,145,1102000


### 4.2 Cost Calculation

Cost represents the operational expenses associated with flying each route for a given
month. For this project, I use a simplified cost model that includes:

- **Variable cost per passenger** (e.g., fuel, catering, handling)
- **Fixed cost per flight** (e.g., crew, aircraft lease, maintenance)
- **Assumed flights per month** (30 flights per route)

**Cost = (Passengers × Cost per Passenger) + (Flights per Month × Fixed Cost per Flight)**

This provides a realistic approximation of airline operating expenses.

In [42]:
COST_PER_PAX = 65
COST_PER_FLIGHT = 18000
FLIGHTS_PER_MONTH = 30

df["cost"] = (df["passengers"] * COST_PER_PAX) + (COST_PER_FLIGHT * FLIGHTS_PER_MONTH)
df[["route", "month", "cost"]].head()

,route,month,cost
72,BOS-CHS,2024-01-01,988500
73,BOS-CHS,2024-02-01,975500
74,BOS-CHS,2024-03-01,1008000
75,BOS-CHS,2024-04-01,1014500
76,BOS-CHS,2024-05-01,1034000


### 4.3 Profit Calculation

Profit measures the financial outcome of each route-month after accounting for both
revenue and operating costs.

**Profit = Revenue − Cost**

Positive profit indicates a financially strong month, while negative profit highlights
periods where the route underperformed or experienced seasonal weakness.

In [43]:
df["profit"] = df["revenue"] - df["cost"]
df[["route", "month", "revenue", "cost", "profit"]].head()

,route,month,revenue,cost,profit
72,BOS-CHS,2024-01-01,890100,988500,-98400
73,BOS-CHS,2024-02-01,837500,975500,-138000
74,BOS-CHS,2024-03-01,972000,1008000,-36000
75,BOS-CHS,2024-04-01,1007400,1014500,-7100
76,BOS-CHS,2024-05-01,1102000,1034000,68000


### 4.4 Profit Margin Calculation

Profit margin expresses profitability as a percentage of revenue, allowing for easier
comparison across routes and months with different scales.

**Profit Margin = Profit ÷ Revenue**

This metric helps identify which routes are most efficient at converting revenue into
profit, regardless of absolute size.

In [44]:
df["profit_margin"] = df["profit"] / df["revenue"]
df[["route", "month", "profit_margin"]].head()

,route,month,profit_margin
72,BOS-CHS,2024-01-01,-0.11
73,BOS-CHS,2024-02-01,-0.16
74,BOS-CHS,2024-03-01,-0.04
75,BOS-CHS,2024-04-01,-0.01
76,BOS-CHS,2024-05-01,0.06


In [45]:
df[["revenue", "cost", "profit", "profit_margin"]].describe()

,revenue,cost,profit,profit_margin
count,120.00,120.00,120.00,120.00
mean,"1,887,531.67","1,215,539.58","671,992.08",0.30
std,"617,064.75","135,364.01","492,769.47",0.18
min,"837,500.00","975,500.00","-138,000.00",-0.16
25%,"1,388,900.00","1,105,500.00","281,537.50",0.20
50%,"1,815,500.00","1,203,000.00","606,000.00",0.34
75%,"2,361,150.00","1,323,250.00","1,052,400.00",0.44
max,"3,182,000.00","1,502,000.00","1,680,000.00",0.54


# Section 5 — Growth Metrics (Lag, MoM Growth)

This section introduces time-series growth metrics commonly used in airline performance
analysis. By computing lagged values and month-over-month (MoM) growth, we can identify
momentum shifts, seasonal patterns, and emerging opportunities across JetBlue’s network.

**Metrics added:**
- **Lagged Revenue, Cost, and Profit** (previous month values)
- **MoM Revenue Growth**
- **MoM Profit Growth**
- **Route-Level Summary Metrics**
- **Opportunity Score** combining growth and profitability signals

These metrics help highlight routes with accelerating demand, improving profitability,
or early signs of underperformance.

### 5.1 Lag Metrics (Previous Month Values)

To calculate month‑over‑month growth, we first need the previous month's values for
each route. Using the `groupby().shift(1)` method, we create lagged features that align
each row with its corresponding prior‑month performance.

Lagged metrics created:
- **lag_passengers** — previous month's passenger count  
- **lag_revenue** — previous month's revenue  
- **lag_profit** — previous month's profit  

The first month of each route will naturally have `NaN` values because there is no
prior month to reference. This is expected and correct.

In [46]:
df["lag_revenue"] = df.groupby("route")["revenue"].shift(1)
df["lag_profit"] = df.groupby("route")["profit"].shift(1)
df["lag_passengers"] = df.groupby("route")["passengers"].shift(1)

### 5.2 Month‑over‑Month Growth

Month‑over‑month (MoM) growth measures how revenue and profit change relative to the
previous month. These metrics help identify momentum shifts, seasonal patterns, and
emerging opportunities across JetBlue’s network.

MoM Growth Formula:
**(Current Month − Previous Month) ÷ Previous Month**

In [47]:
df["revenue_growth"] = (df["revenue"] - df["lag_revenue"]) / df["lag_revenue"]
df["profit_growth"] = (df["profit"] - df["lag_profit"]) / df["lag_profit"]

df[["route", "month", "revenue_growth", "profit_growth"]].head()

,route,month,revenue_growth,profit_growth
72,BOS-CHS,2024-01-01,NaN,NaN
73,BOS-CHS,2024-02-01,-0.06,0.40
74,BOS-CHS,2024-03-01,0.16,-0.74
75,BOS-CHS,2024-04-01,0.04,-0.80
76,BOS-CHS,2024-05-01,0.09,-10.58


### 5.3 Route‑Level Summary

To evaluate overall performance across the full analysis period, each route is
aggregated into a summary table. This includes total revenue, total profit, and average
growth rates. These metrics provide a high‑level view of route strength and help
identify markets with consistent performance or emerging volatility.

In [48]:
route_summary = (
    df.groupby("route")
      .agg(
          total_revenue=("revenue", "sum"),
          total_profit=("profit", "sum"),
          avg_revenue_growth=("revenue_growth", "mean"),
          avg_profit_growth=("profit_growth", "mean")
      )
      .reset_index()
)

route_summary.head()

,route,total_revenue,total_profit,avg_revenue_growth,avg_profit_growth
0,BOS-CHS,12263800,73550,0.03,-1.09
1,BOS-DEN,21519600,7434600,0.02,0.08
2,BOS-MCO,22327800,6695800,0.03,0.12
3,BOS-SEA,28473100,13738100,0.03,0.06
4,FLL-AUS,14371300,1433550,0.02,-1.65


### 5.4 Opportunity Score

The Opportunity Score blends profitability and growth signals into a single metric.
This helps highlight routes that are not only profitable but also accelerating in
performance — a key consideration in airline network planning.

The score is calculated as an equal‑weighted blend of:

- **Average Profit Growth**
- **Average Revenue Growth**

This creates a balanced indicator of both financial strength and momentum.

In [49]:
route_summary["opportunity_score"] = (
    0.5 * route_summary["avg_profit_growth"] +
    0.5 * route_summary["avg_revenue_growth"]
)

route_summary[["route", "opportunity_score"]].head()

,route,opportunity_score
0,BOS-CHS,-0.53
1,BOS-DEN,0.05
2,BOS-MCO,0.07
3,BOS-SEA,0.04
4,FLL-AUS,-0.81


### 5.5 Join Opportunity Score Back to Monthly Data

The Opportunity Score is merged back into the monthly dataset so it can be used in
Tableau visualizations and downstream analysis. This enables route‑level opportunity
signals to be displayed alongside monthly revenue, profit, and growth metrics.

In [50]:
df = df.merge(route_summary[["route", "opportunity_score"]], on="route", how="left")
df.head()

,origin,destination,month,distance_miles,seats,passengers,load_factor,avg_fare,competitor_seats,route,...,revenue,cost,profit,profit_margin,lag_revenue,lag_profit,lag_passengers,revenue_growth,profit_growth,opportunity_score
0,BOS,CHS,2024-01-01,816,8000,6900,0.86,129,3000,BOS-CHS,...,890100,988500,-98400,-0.11,NaN,NaN,NaN,NaN,NaN,-0.53
1,BOS,CHS,2024-02-01,816,8000,6700,0.84,125,3000,BOS-CHS,...,837500,975500,-138000,-0.16,"890,100.00","-98,400.00","6,900.00",-0.06,0.40,-0.53
2,BOS,CHS,2024-03-01,816,8000,7200,0.90,135,3500,BOS-CHS,...,972000,1008000,-36000,-0.04,"837,500.00","-138,000.00","6,700.00",0.16,-0.74,-0.53
3,BOS,CHS,2024-04-01,816,8000,7300,0.91,138,3500,BOS-CHS,...,1007400,1014500,-7100,-0.01,"972,000.00","-36,000.00","7,200.00",0.04,-0.80,-0.53
4,BOS,CHS,2024-05-01,816,8000,7600,0.95,145,4000,BOS-CHS,...,1102000,1034000,68000,0.06,"1,007,400.00","-7,100.00","7,300.00",0.09,-10.58,-0.53


# Section 6 — Export Enriched Dataset for Tableau

With all enrichment, financial metrics, and growth calculations complete, the final
dataset is exported for use in Tableau. This enables interactive visualization of route
performance, profitability, growth trends, and opportunity scoring across JetBlue’s
network.

The exported file includes:
- Aircraft assignments  
- Seat configurations  
- Revenue, cost, profit, and margin metrics  
- Lag features and MoM growth  
- Route‑level opportunity scores  

This dataset serves as the foundation for the Tableau dashboards built in Section 7.

In [51]:
df.to_csv("jetblue_enriched_dataset.csv", index=False)
df.head()


,origin,destination,month,distance_miles,seats,passengers,load_factor,avg_fare,competitor_seats,route,...,revenue,cost,profit,profit_margin,lag_revenue,lag_profit,lag_passengers,revenue_growth,profit_growth,opportunity_score
0,BOS,CHS,2024-01-01,816,8000,6900,0.86,129,3000,BOS-CHS,...,890100,988500,-98400,-0.11,NaN,NaN,NaN,NaN,NaN,-0.53
1,BOS,CHS,2024-02-01,816,8000,6700,0.84,125,3000,BOS-CHS,...,837500,975500,-138000,-0.16,"890,100.00","-98,400.00","6,900.00",-0.06,0.40,-0.53
2,BOS,CHS,2024-03-01,816,8000,7200,0.90,135,3500,BOS-CHS,...,972000,1008000,-36000,-0.04,"837,500.00","-138,000.00","6,700.00",0.16,-0.74,-0.53
3,BOS,CHS,2024-04-01,816,8000,7300,0.91,138,3500,BOS-CHS,...,1007400,1014500,-7100,-0.01,"972,000.00","-36,000.00","7,200.00",0.04,-0.80,-0.53
4,BOS,CHS,2024-05-01,816,8000,7600,0.95,145,4000,BOS-CHS,...,1102000,1034000,68000,0.06,"1,007,400.00","-7,100.00","7,300.00",0.09,-10.58,-0.53


# Section 7 — Tableau Dashboards

The enriched dataset is visualized in Tableau to provide an interactive view of route
performance, aircraft utilization, and network opportunities. These dashboards highlight
key insights across financial, operational, and growth dimensions.


### 7.1 Route‑Level Dashboards

These dashboards provide a high‑level view of route performance across JetBlue’s
network, enabling quick identification of profitable markets, seasonal patterns, and
routes with strong or declining momentum.

Dashboards created:
- **Route Profitability Overview**  
  Visualizes revenue, cost, profit, and margin across all routes and months.

- **Growth & Momentum Analysis**  
  Displays MoM revenue and profit growth, highlighting routes with accelerating or
  declining performance.

- **Opportunity Score Ranking**  
  Ranks routes based on combined profitability and growth signals to identify
  high‑potential markets.


### 7.2 Aircraft‑Level Profitability Dashboards

To visualize aircraft‑level performance, the enriched dataset is imported into Tableau
and used to build a dedicated Fleet Profitability dashboard. This dashboard highlights
how different aircraft types contribute to JetBlue’s financial and operational results,
revealing which fleet segments drive the strongest returns and how aircraft assignment
impacts route‑level profitability.

These visuals help identify high‑performing aircraft types, evaluate efficiency
differences across the fleet, and uncover opportunities for optimized aircraft
deployment.


### 7.3 Key Insights

- Several routes show strong seasonality, with summer months outperforming winter.
- Profitability varies significantly by route, with some markets consistently
  generating losses despite stable demand.
- Revenue growth and passenger growth do not always move together, indicating
  pricing effects and yield management opportunities.
- Certain routes demonstrate strong momentum, suggesting potential for increased
  capacity or frequency adjustments.


# Section 8 — SQL Analysis

This section recreates key analytical steps using SQL to demonstrate proficiency with
database querying and airline‑style performance analysis. The enriched dataset is
loaded into an in‑memory SQLite database, and a series of queries are executed to
analyze route profitability, growth trends, and aircraft‑level performance.

The SQL queries include:
- Route‑level profitability
- Monthly performance trends
- Window functions (LAG)
- Growth calculations
- Top‑performing route‑months
- Aircraft‑level profitability and efficiency metrics

These queries mirror the types of analyses performed by airline network planning and
revenue management teams, and validate that the Python‑based workflow can be replicated
in a SQL environment.


### 8.1 Creating the SQL Table

The enriched dataset is loaded into an in‑memory SQLite database and stored in a table
named `route_performance`. This enables SQL‑based exploration of route profitability,
growth metrics, and aircraft‑level performance using the same data prepared in Python.

By loading the dataset into a SQL table, we can validate analytical logic, replicate
key calculations, and demonstrate proficiency with SQL queries commonly used in airline
network planning and revenue management environments.


In [52]:
# Connect to SQLite and load the enriched dataset into a SQL table
conn = sqlite3.connect("jetblue.db")
df.to_sql("route_performance", conn, if_exists="replace", index=False)


120

### Query 1 — Total Profit by Route

This query calculates total profit for each route, helping identify the strongest and
weakest performers.

In [53]:
query = """
SELECT route,
         SUM(profit) AS total_profit
FROM route_performance
GROUP BY route
ORDER BY total_profit DESC
"""
pd.read_sql_query(query, conn)

,route,total_profit
0,JFK-SFO,15144900
1,JFK-LAX,15072800
2,BOS-SEA,13738100
3,JFK-SAN,12839800
4,BOS-DEN,7434600
5,BOS-MCO,6695800
6,JFK-AUS,5484850
7,FLL-EWR,2721100
8,FLL-AUS,1433550
9,BOS-CHS,73550


### Query 2 — Monthly Profit Trend for a Selected Route

This query retrieves month-by-month profit for a specific route (e.g., JFK-LAX, BOS-SEA),
mirroring the trend analysis performed earlier in Python.

In [54]:
query = """
SELECT route, month, profit
FROM route_performance
WHERE route = 'JFK-LAX' OR route = 'BOS-SEA'
ORDER BY route;
"""
pd.read_sql_query(query, conn)

,route,month,profit
0,BOS-SEA,2024-01-01 00:00:00,881000
1,BOS-SEA,2024-02-01 00:00:00,930000
2,BOS-SEA,2024-03-01 00:00:00,1051200
3,BOS-SEA,2024-04-01 00:00:00,1087500
4,BOS-SEA,2024-05-01 00:00:00,1188000
5,BOS-SEA,2024-06-01 00:00:00,1364000
6,BOS-SEA,2024-07-01 00:00:00,1530000
7,BOS-SEA,2024-08-01 00:00:00,1455000
8,BOS-SEA,2024-09-01 00:00:00,990000
9,BOS-SEA,2024-10-01 00:00:00,1035900


### Query 3 — Using Window Functions to Compute Previous Month Revenue

This query demonstrates SQL window functions by calculating the previous month's
revenue for each route, similar to the `shift()` logic used in Python.

In [55]:
query = """
SELECT
    route,
    month,
    revenue,
    LAG(revenue, 1) OVER (PARTITION BY route ORDER BY month) AS revenue_prev
FROM route_performance;
"""
pd.read_sql_query(query, conn)

,route,month,revenue,revenue_prev
0,BOS-CHS,2024-01-01 00:00:00,890100,NaN
1,BOS-CHS,2024-02-01 00:00:00,837500,"890,100.00"
2,BOS-CHS,2024-03-01 00:00:00,972000,"837,500.00"
3,BOS-CHS,2024-04-01 00:00:00,1007400,"972,000.00"
4,BOS-CHS,2024-05-01 00:00:00,1102000,"1,007,400.00"
...,...,...,...,...
115,JFK-SFO,2024-08-01 00:00:00,2970000,"3,060,000.00"
116,JFK-SFO,2024-09-01 00:00:00,2415600,"2,970,000.00"
117,JFK-SFO,2024-10-01 00:00:00,2480000,"2,415,600.00"
118,JFK-SFO,2024-11-01 00:00:00,2284800,"2,480,000.00"


### Query 4 — Revenue Growth Calculation in SQL

This query computes month-over-month revenue growth using SQL expressions and window
functions.

In [56]:
query = """
SELECT
    route,
    month,
    revenue,
    LAG(revenue, 1) OVER (PARTITION BY route ORDER BY month) AS revenue_prev,
    (revenue - LAG(revenue, 1) OVER (PARTITION BY route ORDER BY month))
        / LAG(revenue, 1) OVER (PARTITION BY route ORDER BY month) AS revenue_growth
FROM route_performance;
"""
pd.read_sql_query(query, conn)

,route,month,revenue,revenue_prev,revenue_growth
0,BOS-CHS,2024-01-01 00:00:00,890100,NaN,NaN
1,BOS-CHS,2024-02-01 00:00:00,837500,"890,100.00",0.00
2,BOS-CHS,2024-03-01 00:00:00,972000,"837,500.00",0.00
3,BOS-CHS,2024-04-01 00:00:00,1007400,"972,000.00",0.00
4,BOS-CHS,2024-05-01 00:00:00,1102000,"1,007,400.00",0.00
...,...,...,...,...,...
115,JFK-SFO,2024-08-01 00:00:00,2970000,"3,060,000.00",0.00
116,JFK-SFO,2024-09-01 00:00:00,2415600,"2,970,000.00",0.00
117,JFK-SFO,2024-10-01 00:00:00,2480000,"2,415,600.00",0.00
118,JFK-SFO,2024-11-01 00:00:00,2284800,"2,480,000.00",0.00


### Query 5 — Top 5 Most Profitable Route-Months

This query identifies the highest-profit route-month combinations, useful for spotting
seasonal peaks or standout markets.

In [57]:
query = """
SELECT route, month, profit
FROM route_performance
ORDER BY profit DESC
LIMIT 5;
"""
pd.read_sql_query(query, conn)

,route,month,profit
0,JFK-LAX,2024-07-01 00:00:00,1680000
1,JFK-SFO,2024-07-01 00:00:00,1636000
2,JFK-LAX,2024-08-01 00:00:00,1591500
3,JFK-SFO,2024-08-01 00:00:00,1552500
4,JFK-LAX,2024-12-01 00:00:00,1548000


### Query 6 — Total Profit by Aircraft Type

This query aggregates total profit at the aircraft level to compare financial
performance across JetBlue’s fleet types. It highlights which aircraft generate the
strongest overall contribution to route profitability.

In [58]:
query = """
SELECT aircraft_type,
    SUM(profit) AS total_profit,
    AVG(profit) AS avg_profit_per_month
FROM route_performance
GROUP BY aircraft_type
ORDER BY total_profit DESC;
"""
pd.read_sql_query(query, conn)

,aircraft_type,total_profit,avg_profit_per_month
0,A321 Mint,43057500,"1,196,041.67"
1,A320,19615250,"544,868.06"
2,A321,13738100,"1,144,841.67"
3,A220,4228200,"117,450.00"


### Query 7 — Average Load Proxy by Aircraft Type

This query computes a simplified load factor proxy by comparing average passengers
to the configured seat count for each aircraft type. While not a true load factor,
it provides a useful approximation of demand relative to capacity.

In [59]:
query = """
SELECT
    aircraft_type,
    AVG(passengers) AS avg_passengers,
    AVG(seats_configured) AS avg_seats,
    AVG(passengers) * 1.0 / AVG(seats_configured) AS load_factor_proxy
FROM route_performance
GROUP BY aircraft_type
ORDER BY load_factor_proxy DESC;
"""
pd.read_sql_query(query, conn)

,aircraft_type,avg_passengers,avg_seats,load_factor_proxy
0,A321 Mint,"12,808.33",159.00,80.56
1,A320,"10,209.72",162.00,63.02
2,A220,"8,097.22",140.00,57.84
3,A321,"10,583.33",200.00,52.92


### Query 8 — Profit per Seat by Aircraft Type

This query normalizes total profit by the number of configured seats for each aircraft
type. Profit per seat provides a capacity-adjusted view of financial efficiency across
the fleet.

In [60]:
query = """
SELECT
    aircraft_type,
    SUM(profit) AS total_profit,
    SUM(seats_configured) AS total_seats_configured,
    SUM(profit) * 1.0 / SUM(seats_configured) AS profit_per_seat_proxy
FROM route_performance
GROUP BY aircraft_type
ORDER BY profit_per_seat_proxy DESC;
"""
pd.read_sql_query(query, conn)

,aircraft_type,total_profit,total_seats_configured,profit_per_seat_proxy
0,A321 Mint,43057500,5724,"7,522.27"
1,A321,13738100,2400,"5,724.21"
2,A320,19615250,5832,"3,363.38"
3,A220,4228200,5040,838.93


### Query 9 — Monthly Profit Trend by Aircraft Type

This query shows month-by-month profit aggregated by aircraft type. It reveals
seasonality patterns and helps compare how different fleet types perform throughout
the year.

In [61]:
query = """
SELECT
    aircraft_type,
    month,
    SUM(profit) AS monthly_profit
FROM route_performance
GROUP BY aircraft_type, month
ORDER BY aircraft_type, month;
"""
pd.read_sql_query(query, conn)

,aircraft_type,month,monthly_profit
0,A220,2024-01-01 00:00:00,-1000
1,A220,2024-02-01 00:00:00,-124800
2,A220,2024-03-01 00:00:00,212800
3,A220,2024-04-01 00:00:00,291500
4,A220,2024-05-01 00:00:00,538800
5,A220,2024-06-01 00:00:00,755000
6,A220,2024-07-01 00:00:00,923500
7,A220,2024-08-01 00:00:00,843900
8,A220,2024-09-01 00:00:00,111700
9,A220,2024-10-01 00:00:00,197000


### Query 10 — Aircraft Type Mix by Route

This query identifies which aircraft types operate each route in the dataset. While
each route in this project uses a single assigned aircraft type, this query mirrors
real-world fleet assignment analysis where multiple aircraft may serve the same market.

In [62]:
query = """
SELECT
    route,
    aircraft_type,
    COUNT(*) AS months_operated
FROM route_performance
GROUP BY route, aircraft_type
ORDER BY route
"""
pd.read_sql_query(query, conn)

,route,aircraft_type,months_operated
0,BOS-CHS,A220,12
1,BOS-DEN,A320,12
2,BOS-MCO,A320,12
3,BOS-SEA,A321,12
4,FLL-AUS,A220,12
5,FLL-EWR,A220,12
6,JFK-AUS,A320,12
7,JFK-LAX,A321 Mint,12
8,JFK-SAN,A321 Mint,12
9,JFK-SFO,A321 Mint,12


### 8.2 Summary

These SQL queries replicate the core analytical steps performed in Python, including
profitability analysis, growth calculations, window functions, and aircraft‑level
performance evaluation. By validating the workflow in SQL, this section demonstrates
the ability to translate analytical logic across tools and reinforces proficiency with
query‑driven analysis commonly used in airline network planning and revenue management
environments.


# Section 9 — Conclusion & Recommendations

This analysis provides a comprehensive view of JetBlue’s route performance across
financial, operational, and growth dimensions. By integrating aircraft type,
seat configuration, profitability metrics, and month-over-month trends, the project
mirrors real-world airline network planning workflows.

### Key Takeaways
- Premium transcontinental routes operated with A321 Mint aircraft generate strong
  revenue and competitive profit margins.
- A220-operated routes show efficient right-sized capacity, with solid load factor
  proxies despite smaller gauge.
- Profitability varies significantly by route, with some markets showing strong
  seasonality and others demonstrating consistent performance.
- Growth metrics reveal momentum shifts that can inform schedule adjustments or
  targeted pricing strategies.

### Recommendations
- **Increase focus on A321 Mint markets**, which show strong revenue and premium
  demand characteristics.
- **Monitor A220 routes for growth opportunities**, as these aircraft provide
  flexibility and efficiency on thinner markets.
- **Investigate underperforming routes** with persistently negative profit margins
  to determine whether schedule, pricing, or aircraft assignment adjustments are
  warranted.
- **Leverage growth trends** to identify markets with accelerating demand for
  potential frequency increases.